#Tools

In [ ]:
!rm -rf sample_data # remove default data

In [ ]:
def nGPUs(): # CPU cores, adopted from: https://stackoverflow.com/a/63833950
  import torch
  if torch.cuda.is_available():
      from numba import cuda
      devInfo = {
          (2,0) : 32,
          (2,1) : 48,
          (3,0) : 192,
          (3,5) : 192,
          (3,7) : 192,
          (5,0) : 128,
          (5,2) : 128,
          (6,0) : 64,
          (6,1) : 128,
          (7,0) : 64,
          (7,5) : 64,
          (8,0) : 64,
          (8,6) : 128,
          (8,9) : 128,
          (9,0) : 128
          }
      GPU = cuda.get_current_device()
      iGPU = getattr(GPU, 'MULTIPROCESSOR_COUNT')
      doGPU = GPU.compute_capability
      per_iCPU = devInfo.get(doGPU)
      nGPU = per_iCPU*iGPU
      return nGPU
  else:
      return 0
#-------------------------------------------------------------------------------
def nCPUs(): # CPU cores
  import multiprocessing
  nCPU = multiprocessing.cpu_count()
  return nCPU
#-------------------------------------------------------------------------------


In [ ]:
def MyApps(appname, flags): #flags = ['buildit', 'loadit', 'saveit', 'makeitready', 'usage'].
  # by default saves/packs all new builds
  import os
  nCPU = nCPUs()
  nGPU = nGPUs()
  match appname:
    #---------------------------------------------------------------------------
    case 'lammps':
      if 'buildit' == flags:
        print('building lammps from source')
        !git clone -b release https://github.com/lammps/lammps.git lammps
        %cd lammps
        !mkdir build
        %cd build
        !cmake -D BUILD_MPI=ON -D BUILD_GPU=ON -D PKG_USER-MEAC=ON -D PKG_MANYBODY=ON -D PKG_MOLECULE=ON -D PKG_PYTHON=ON -D PKG_MC=ON  -D BUILD_DIELECTRIC=ON -D BUILD_DIPOLE=ON -D BUILD_EFF=ON -D BUILD_EXTRA-COMPUTE=ON -D BUILD_EXTRA-DUMP=ON -D BUILD_EXTRA-FIX=ON -D BUILD_EXTRA-MOLECULE=ON -D BUILD_EXTRA-PAIR=ON -D BUILD_GRANULAR=ON -D BUILD_KSPACE=ON -D BUILD_LATBOLTZ=ON -D BUILD_ML-HDNNP=ON -D BUILD_ML-IAP=ON -D BUILD_ML-POD=ON -D BUILD_ML-SNAP=ON -D BUILD_MOLECULE=ON -D BUILD_ORIENT=ON -D BUILD_QEQ=ON -D BUILD_QMMM=ON -D BUILD_SPIN=ON -D BUILD_VORONOI=ON ../cmake
        !cmake --build .
        %cd ../../
        print('completed building lammps from source')
        MyApps('lammps', 'saveit')
      if 'saveit' == flags:
        print('packing lammps is requested, creating pack lammps.zip')
        if os.path.exists('lammps.zip'):
          !rm lammps.zip
        else:
          if os.path.exists('lammps'):
            !zip lammps.zip lammps
            print('lammps.zip is created')
          else:
            print('build first')
      if 'loadit' == flags:
        print('preparing lammps from a previous built')
        print('looking for lammps.zip to unpack')
        if os.path.exists('lammps.zip'):
            print('lammps.zip is available, unzipping')
            !unzip lammps.zip -d /content/
            print('completed unzipping lammps.zip')
        else:
            print('lammps.zip is not available')
      if 'makeitready' == flags:
        print('adding lammps to path')
        os.environ["OMP_NUM_THREADS"] = f'{nCPU}'
        os.environ["MKL_NUM_THREADS"] = f'{nCPU}'
        os.environ["PATH"] = "/content/lammps/build:"+os.environ["PATH"]
        print('completed adding lammps to path')
        print('lammps is ready')
      if 'usage' == flags:
        print('Usage: !mpirun -np 2 --allow-run-as-root /content/lammps/build/lmp -in /content/lammps/examples/crack/in.crack')
      if not (flags in ['buildit', 'loadit', 'saveit', 'makeitready', 'usage']):
        print('flag not found. use: buildit, loadit, saveit, makeitready, usage')
    #---------------------------------------------------------------------------
    case 'abinit':
      print('abinit requires libxc, looking for libxc')
      if os.path.exists('libxc'):
        MyApps('libxc', 'makeitready')
      elif os.path.exists('libxc.zip'):
        MyApps('libxc', 'loadit')
        MyApps('libxc', 'makeitready')
      else:
        print('found neither libxc.zip nor libxc, building from source')
        MyApps('libxc', 'builtit')
        MyApps('libxc', 'saveit')
      print('libxc is ready, starting with abinit')
      #-------------------------------------------------------------------------
      if 'buildit' == flags:
        print('building abinit from source')
        #!git clone https://github.com/abinit/abinit.git abinit
        !wget https://www.abinit.org/sites/default/files/packages/abinit-9.10.5.tar.gz
        !tar xzf abinit-9.10.5.tar.gz
        !mv abinit-9.10.5 abinit
        !rm abinit-9.10.5.tar.gz
        %cd abinit/
        %mkdir build
        %cd build
        !.././configure
        !make
        !sudo make install
        %cd ../../
        print('completed building abinit from source')
        MyApps('abinit', 'saveit')
      if 'saveit' in flags:
        print('packing abinit is requested, creating pack abinit.zip')
        if os.path.exists('abinit.zip'):
          !rm abinit.zip
        else:
          if os.path.exists('abinit'):
            !zip abinit.zip abinit
            print('abinit.zip is created')
          else:
            print('build first')
      if 'loadit' == flags:
        print('preparing abinit from a previous built')
        print('looking for abinit.zip to unpack')
        if os.path.exists('abinit.zip'):
            print('abinit.zip is available, unzipping')
            !unzip abinit.zip -d /content/
            print('completed unzipping abinit.zip')
        else:
            print('abinit.zip is not available')
      if 'makeitready' == flags:
        print('adding abinit to path')
        os.environ["OMP_NUM_THREADS"] = f'{nCPU}'
        os.environ["MKL_NUM_THREADS"] = f'{nCPU}'
        os.environ['PATH'] = "/content/abinit/bin:"+os.environ['PATH']
        print('abinit is ready.')
      if 'usage' in flags:
        print('Usage: !mpirun -np 2 --allow-run-as-root abinit')
        print('For pseudopotentials see: https://www.abinit.org/psp-tables')
      if not (flags in ['buildit', 'loadit', 'saveit', 'makeitready', 'usage']):
        print('flag not found. use: buildit, loadit, saveit, makeitready, usage')
    #---------------------------------------------------------------------------
    case 'libxc':
      if 'buildit' == flags:
        !apt-get -y install gfortran
        !apt-get -y install build-essential "liblapack*"
        !apt-get -y install build-essential "libblas*"
        !apt-get -y install build-essential "libnetcdf*"
        !apt-get -y install build-essential "libxml2*"
        !apt-get -y install build-essential "libfftw3*"
        !apt-get -y install "openmpi*"
        !apt-get -y install "libopenmpi*"
        # for basic install use libxckxc = 0 <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
        libxckxc = 0
        if libxckxc == 1:
          for _ in range(10):
            print('A VERY LONG PROCESS STARTING ....')
          !wget https://www.tddft.org/programs/libxc/down/6.2.2/libxc-6.2.2.tar.gz
          !tar -xzf libxc-6.2.2.tar.gz
          !mv libxc-6.2.2 libxc
          !rm libxc-6.2.2.tar.gz
          %cd libxc/
          !./configure --prefix=/content/libxc --enable-kxc # Otherwise Abinit will not be able to provide nonlinear response properties (Raman intensities, ...).
          !make
          !sudo make install
          %cd ..
        else:
          !apt-get -y install build-essential "libxc*"
          !apt-get -y install build-essential "libxc-dev"
        print('completed building libxc from source')
        MyApps('libxc', 'saveit')
      if 'saveit' == flags:
        print('packing libxc is requested, creating pack libxc.zip')
        if os.path.exists('libxc.zip'):
          !rm libxc.zip
        else:
          if os.path.exists('libxc'):
            !zip libxc.zip libxc
            print('libxc.zip is created')
          else:
            print('build first')
      if 'loadit' == flags:
        print('preparing libxc from a previous built')
        print('looking for libxc.zip to unpack')
        if os.path.exists('libxc.zip'):
          print('libxc.zip is available, unzipping')
          !unzip libxc.zip -d /content/
        else:
          print('libxc.zip is not available.')
        if os.path.exists('libxc'):
          print('libxc available')
        else:
          print('libxc not available')
      if 'makeitready' == flags:
        print('separate activation not needed, called internally.')

      if not (flags in ['buildit', 'loadit', 'saveit', 'makeitready', 'usage']):
        print('flag not found. use: buildit, loadit, saveit, makeitready, usage')

# Building ...

In [ ]:
print('CPU cores: ', nCPUs())
print('GPU cores: ', nGPUs())

CPU cores:  2
GPU cores:  0


In [ ]:
MyApps('lammps', 'buildit')
MyApps('lammps', 'makeitready')
MyApps('lammps', 'usage')

In [ ]:
MyApps('libxc', 'buildit')
MyApps('libxc', 'makeitready')
MyApps('libxc', 'usage')

In [ ]:
MyApps('abinit', 'buildit')
MyApps('abinit', 'makeitready')
MyApps('abinit', 'usage')

# Getting things ready for ABINIT

In [ ]:
!wget https://www.abinit.org/ATOMICDATA/JTH-PBE-atomicdata.tar.gz
!tar -xzf JTH-PBE-atomicdata.tar.gz
!wget https://www.abinit.org/ATOMICDATA/JTH-LDA-atomicdata.tar.gz
!tar -xzf JTH-LDA-atomicdata.tar.gz
!mv JTH-PBE-atomicdata-1.1 PBE
!mv JTH-LDA-atomicdata-1.1 LDA
!rm -rf *.gz
!mkdir PAW
!mv PBE PAW/
!mv LDA PAW/
!mkdir PSP
!wget http://www.pseudo-dojo.org/pseudos/nc-sr_pbe_standard_psp8.tgz
!tar -xzf nc-sr_pbe_standard_psp8.tgz
!mv pbe_s_sr/* PSP/
!rm *.tgz
!rm -rf pbe_s_sr

#Testing

In [ ]:
import os
os.environ['ABI_HOME'] = 'abinit'
os.environ['PATH'] = os.environ['ABI_HOME'] + '/src/98_main/' + ':' + os.environ['PATH']
os.environ['ABI_TESTS'] = os.environ['ABI_HOME'] + '/tests/'
os.environ['ABI_PSPDIR'] = os.environ['ABI_TESTS'] + 'Psps_for_tests/'

In [ ]:
!mkdir desktop

In [ ]:
!echo $ABI_HOME

In [ ]:
%cd desktop

In [ ]:
!cp -avr ../abinit/tests/tutorial/Input .

In [ ]:
!ls

In [ ]:
%cd Input

In [ ]:
%mkdir Work

In [ ]:
%cd Work

In [ ]:
!cp ../tbase1_1.abi .

In [ ]:
!cat tbase1_1.abi

In [ ]:
!abinit tbase1_1.abi >& log &

In [ ]:
!ls

In [ ]:
!cat log

In [ ]:
#!mpirun -np 2 --allow-run-as-root /content/lammps/build/lmp -in /content/lammps/examples/crack/in.crack
#!mpirun -np 2 --allow-run-as-root abinit

In [ ]:
!mpirun --oversubscribe -np 2 --allow-run-as-root /content/lammps/build/lmp -in /content/lammps/examples/crack/in.crack

In [ ]:
!mpirun --oversubscribe -np 2 --allow-run-as-root abinit

In [ ]:
!ls